In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

# Mount Drive for saving outputs
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

# Install pinned kaggle + set credentials
!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "KGAT_17c43012d9e77edf2c183a25acb1489b"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Download + unzip dataset
os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Link data from Kaggle download
os.system('ln -sf /content/data/train.parquet datasets/train.parquet')
os.system('ln -sf /content/data/valid.parquet datasets/valid.parquet')

# Verify
assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print(f"GPU: {os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()}")
print("Ready!")

In [ ]:
# Cell 2: Kill Test — gru_large_v1 (h=256) seeds 42-43
# Expected: ~8-10 min per seed (60 epochs max, early stopping ~30-40)
# PASS threshold: mean val > 0.270 (current GRU mean is 0.262-0.263)
import os
os.chdir("/content/competition_package")

for seed in [42, 43]:
    print(f"\n{'='*60}")
    print(f'Training gru_large_v1 (h=256) seed {seed}')
    print(f"{'='*60}")
    os.system(
        f'python -u scripts/train.py '
        f'--config configs/gru_large_v1.yaml '
        f'--seed {seed} --device cuda'
    )

print('\nKill test h=256 done!')

In [ ]:
# Cell 3: Evaluate kill test results
import os, torch, glob
os.chdir("/content/competition_package")

results = []
for pt in sorted(glob.glob('logs/gru_large_v1_seed*.pt')):
    ckpt = torch.load(pt, map_location='cpu', weights_only=False)
    score = ckpt.get('best_score', 0.0)
    epoch = ckpt.get('best_epoch', 0)
    name = os.path.basename(pt)
    results.append((name, float(score), epoch))
    print(f'{name}: val={score:.4f} (best epoch {epoch})')

if results:
    scores = [s for _, s, _ in results]
    mean_score = sum(scores) / len(scores)
    print(f'\n--- Kill Test Result ---')
    print(f'Mean val: {mean_score:.4f}')
    print(f'Current GRU baseline mean: ~0.2620')
    print(f'Delta: {mean_score - 0.2620:+.4f}')
    if mean_score > 0.275:
        print('STRONG PASS! Proceed to 5-seed expansion.')
    elif mean_score > 0.270:
        print('PASS! Proceed to 5-seed expansion.')
    elif mean_score > 0.265:
        print('MARGINAL. Consider testing h=256 with old recipe to isolate effect.')
    else:
        print('FAIL. Try h=256 with current recipe (no cosine/EMA) to isolate.')

In [ ]:
# Cell 4: Full expansion — gru_large_v1 seeds 44-48 (5 more seeds)
# ONLY RUN IF KILL TEST PASSED (mean > 0.270)
import os
os.chdir("/content/competition_package")

for seed in range(44, 49):
    print(f"\n{'='*60}")
    print(f'Training gru_large_v1 (h=256) seed {seed}')
    print(f"{'='*60}")
    os.system(
        f'python -u scripts/train.py '
        f'--config configs/gru_large_v1.yaml '
        f'--seed {seed} --device cuda'
    )

print('\nExpansion done: h=256 seeds 44-48')

In [ ]:
# Cell 5: Kill Test — gru_xl_v1 (h=384) seeds 42-43
# ONLY RUN IF h=256 PASSED and you have time remaining
# Expected: ~15-20 min per seed
import os
os.chdir("/content/competition_package")

for seed in [42, 43]:
    print(f"\n{'='*60}")
    print(f'Training gru_xl_v1 (h=384) seed {seed}')
    print(f"{'='*60}")
    os.system(
        f'python -u scripts/train.py '
        f'--config configs/gru_xl_v1.yaml '
        f'--seed {seed} --device cuda'
    )

print('\nKill test h=384 done!')

In [ ]:
# Cell 5: Evaluate all results (h=256 + h=384)
import os, torch, glob
os.chdir(REPO)

for config_name in ['gru_large_v1', 'gru_xl_v1']:
    pts = sorted(glob.glob(f'logs/{config_name}_seed*.pt'))
    if not pts:
        continue
    print(f'\n--- {config_name} ---')
    scores = []
    for pt in pts:
        ckpt = torch.load(pt, map_location='cpu', weights_only=False)
        score = float(ckpt.get('best_score', 0.0))
        epoch = ckpt.get('best_epoch', 0)
        name = os.path.basename(pt)
        scores.append(score)
        print(f'  {name}: val={score:.4f} (epoch {epoch})')
    print(f'  Mean: {sum(scores)/len(scores):.4f}, Best: {max(scores):.4f}, Worst: {min(scores):.4f}')

In [ ]:
# Cell 7: Strip checkpoints + copy normalizers + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs('logs/slim', exist_ok=True)

for pt in sorted(glob.glob('logs/gru_large_v1_*.pt') + glob.glob('logs/gru_xl_v1_*.pt')):
    if '_epoch' in pt:
        continue  # Skip periodic checkpoints, only keep best
    ckpt = torch.load(pt, map_location='cpu', weights_only=False)
    slim = {
        'model_state_dict': ckpt['model_state_dict'],
        'config': ckpt.get('config', {}),
        'best_score': ckpt.get('best_score', None),
    }
    out = f'logs/slim/{os.path.basename(pt)}'
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f'{os.path.basename(pt)}: {orig:.1f}MB -> {new:.1f}MB')

for npz in sorted(glob.glob('logs/normalizer_gru_large*.npz') + glob.glob('logs/normalizer_gru_xl*.npz')):
    shutil.copy(npz, f'logs/slim/{os.path.basename(npz)}')
    print(f'Copied {os.path.basename(npz)}')

print(f'\n--- logs/slim/ contents ({len(os.listdir("logs/slim"))} files) ---')
total_mb = 0
for f in sorted(os.listdir('logs/slim')):
    sz = os.path.getsize(f'logs/slim/{f}') / 1e6
    total_mb += sz
    print(f'  {f}: {sz:.1f}MB')
print(f'  Total: {total_mb:.1f}MB')

# Zip for download
shutil.make_archive('/content/gru_large_kill_test', 'zip', '/content/competition_package/logs/slim')
sz = os.path.getsize('/content/gru_large_kill_test.zip') / 1e6
print(f'\ngru_large_kill_test.zip: {sz:.1f}MB')

# Save to Drive for persistence
shutil.copy('/content/gru_large_kill_test.zip', '/content/drive/MyDrive/wunderfund/gru_large_kill_test.zip')
print('Saved to Drive: MyDrive/wunderfund/gru_large_kill_test.zip')

In [ ]:
# Cell 8: Training curve comparison (cosine vs plateau)
import json, glob, os
os.chdir("/content/competition_package")

for hist_file in sorted(glob.glob('logs/training_history_gru_large*.json') + glob.glob('logs/training_history_gru_xl*.json')):
    with open(hist_file) as f:
        hist = json.load(f)
    name = os.path.basename(hist_file).replace('training_history_', '').replace('.json', '')
    scores = [s['avg'] for s in hist['val_scores']]
    lrs = hist['learning_rates']
    print(f'\n--- {name} ---')
    print(f'  Epochs: {len(scores)}, Best: {max(scores):.4f} at epoch {scores.index(max(scores))+1}')
    print(f'  LR range: {min(lrs):.2e} to {max(lrs):.2e}')
    print(f'  Last 5 val scores: {["{:.4f}".format(s) for s in scores[-5:]]}')